# Tutorial 04: Context and Memory - Personalized Travel Assistant

##  Learning Objectives
By the end of this notebook, you will:
- Understand the difference between threads (conversation history) and memory (user knowledge)
- Create custom Context Providers to add persistent memory
- Extract and store user preferences automatically
- Build personalized agents that remember user details across sessions
- Serialize and restore memory state

##  Key Concepts

### Threads vs. Memory: What's the Difference?

**Threads (from Tutorial 03):**
- Store conversation history ("I said X, you said Y")
- Temporary: Good for a single conversation session
- Example: "User asked about Paris, then asked about weather there"

**Memory (this tutorial):**
- Store facts about the user ("User prefers beach vacations")
- Persistent: Survives across sessions and threads
- Example: "User's name is Sarah, budget-conscious, vegetarian"

### Context Providers

A **Context Provider** is a component that:
1. **Listens** to conversations (via `invoked()` method)
2. **Extracts** important information (user preferences, facts)
3. **Provides context** before each AI call (via `invoking()` method)
4. **Persists** across sessions (via `serialize()` method)

```python
class CustomMemory(ContextProvider):
    async def invoked(...):
        # Called AFTER agent responds
        # Extract info from conversation
    
    async def invoking(...):
        # Called BEFORE agent responds
        # Provide remembered context
    
    def serialize(...):
        # Save memory state
```

---

## Step 1: Setup and Imports

In [ ]:
import asyncio
import json
from collections.abc import MutableSequence, Sequence
from typing import Annotated, Any
from random import randint, choice

from agent_framework import (
    ChatAgent,
    ChatClientProtocol,
    ChatMessage,
    ChatOptions,
    Context,
    ContextProvider,
)
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()
print(" Imports successful!")

 Imports successful!


## Step 2: The Problem - Agents Without Memory

Let's first see the limitation: even with threads, agents don't remember user details across sessions.

In [2]:
async def agent_without_memory():
    """
    Demonstrates that threads remember conversation but not user facts.
    """
    print("=== Agent Without Memory (Thread Only) ===")
    print("Threads remember the conversation, but not facts about the user.\n")
    
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(async_credential=credential),
            name="BasicTravelAgent",
            instructions="You are a travel assistant. Help users plan trips.",
        ) as agent,
    ):
        # Session 1: User shares preferences
        print("--- Session 1 ---")
        thread1 = agent.get_new_thread()
        
        response1 = await agent.run("My name is Sarah and I love beach vacations.", thread=thread1)
        print(f"User: My name is Sarah and I love beach vacations.")
        print(f"Agent: {response1.text}\n")
        
        # Session 2: NEW thread (simulating new session)
        print("--- Session 2 (New Thread) ---")
        thread2 = agent.get_new_thread()
        
        response2 = await agent.run("Can you recommend a destination?", thread=thread2)
        print(f"User: Can you recommend a destination?")
        print(f"Agent: {response2.text}\n")
        
        print(" Problem: Agent forgot Sarah's name and beach preference!")
        print("   Even if we reused thread1, the preference isn't 'remembered' as a fact.\n")

await agent_without_memory()

=== Agent Without Memory (Thread Only) ===
Threads remember the conversation, but not facts about the user.

--- Session 1 ---
User: My name is Sarah and I love beach vacations.
Agent: Hi Sarah! That sounds wonderful—beach vacations are so relaxing and fun. To help you plan your next trip, could you tell me a bit more about your preferences? For example:

- Are you interested in destinations within a certain country or region?
- Do you prefer quiet, secluded beaches or lively ones with lots of activities?
- Are you traveling solo, with family, or with friends?
- Any particular amenities or activities you love? (Snorkeling, water sports, luxury resorts, local food, etc.)
- What time of year are you planning to travel?

Once I have a bit more info, I can suggest some amazing beach destinations and help you start planning an unforgettable getaway!

--- Session 2 (New Thread) ---
User: My name is Sarah and I love beach vacations.
Agent: Hi Sarah! That sounds wonderful—beach vacations are s

## Step 3: Define User Profile Model

First, let's define what we want to remember about users.

In [3]:
class TravelProfile(BaseModel):
    """User's travel preferences and profile."""
    
    name: str | None = Field(None, description="User's name")
    preferred_destinations: list[str] = Field(
        default_factory=list,
        description="Types of destinations user prefers (beach, mountain, city, etc.)"
    )
    budget_level: str | None = Field(
        None,
        description="Budget level: budget, moderate, or luxury"
    )
    dietary_restrictions: list[str] = Field(
        default_factory=list,
        description="Dietary restrictions (vegetarian, vegan, halal, kosher, etc.)"
    )
    interests: list[str] = Field(
        default_factory=list,
        description="Travel interests (history, food, adventure, relaxation, etc.)"
    )

print(" TravelProfile model defined")
print("\nExample profile:")
example = TravelProfile(
    name="Sarah",
    preferred_destinations=["beach", "tropical"],
    budget_level="moderate",
    dietary_restrictions=["vegetarian"],
    interests=["relaxation", "food"]
)
print(example.model_dump_json(indent=2))

 TravelProfile model defined

Example profile:
{
  "name": "Sarah",
  "preferred_destinations": [
    "beach",
    "tropical"
  ],
  "budget_level": "moderate",
  "dietary_restrictions": [
    "vegetarian"
  ],
  "interests": [
    "relaxation",
    "food"
  ]
}


## Step 4: Create a Context Provider with Memory

Now let's build our memory component!

In [4]:
class TravelMemory(ContextProvider):
    """
    A Context Provider that remembers user travel preferences.
    
    This class:
    1. Listens to conversations and extracts user info
    2. Provides remembered context before each agent response
    3. Persists user profile across sessions
    """
    
    def __init__(
        self,
        chat_client: ChatClientProtocol,
        profile: TravelProfile | None = None,
        **kwargs: Any
    ):
        """Initialize the memory provider.
        
        Args:
            chat_client: The chat client to use for information extraction
            profile: Existing profile to restore, or None for new user
            **kwargs: Additional args to create TravelProfile from dict
        """
        self._chat_client = chat_client
        
        if profile:
            self.profile = profile
        elif kwargs:
            self.profile = TravelProfile.model_validate(kwargs)
        else:
            self.profile = TravelProfile()
    
    async def invoked(
        self,
        request_messages: ChatMessage | Sequence[ChatMessage],
        response_messages: ChatMessage | Sequence[ChatMessage] | None = None,
        invoke_exception: Exception | None = None,
        **kwargs: Any,
    ) -> None:
        """
        Called AFTER the agent responds.
        Extract user information from the conversation.
        """
        # Get user messages only
        if isinstance(request_messages, ChatMessage):
            request_messages = [request_messages]
        
        user_messages = [
            msg for msg in request_messages
            if hasattr(msg, "role") and msg.role.value == "user"
        ]
        
        if not user_messages:
            return
        
        try:
            # Use AI to extract structured information
            result = await self._chat_client.get_response(
                messages=request_messages,  # type: ignore
                chat_options=ChatOptions(
                    instructions="""
                    Extract the user's travel profile from the message if present.
                    Look for: name, destination preferences, budget level, dietary restrictions, interests.
                    Only update fields that are explicitly mentioned.
                    Return nulls/empty lists for fields not mentioned.
                    """,
                    response_format=TravelProfile,
                ),
            )
            
            # Update profile with extracted data (only non-empty fields)
            if result.value and isinstance(result.value, TravelProfile):
                extracted = result.value
                
                if extracted.name:
                    self.profile.name = extracted.name
                
                if extracted.preferred_destinations:
                    # Add new preferences, avoid duplicates
                    for dest in extracted.preferred_destinations:
                        if dest not in self.profile.preferred_destinations:
                            self.profile.preferred_destinations.append(dest)
                
                if extracted.budget_level:
                    self.profile.budget_level = extracted.budget_level
                
                if extracted.dietary_restrictions:
                    for restriction in extracted.dietary_restrictions:
                        if restriction not in self.profile.dietary_restrictions:
                            self.profile.dietary_restrictions.append(restriction)
                
                if extracted.interests:
                    for interest in extracted.interests:
                        if interest not in self.profile.interests:
                            self.profile.interests.append(interest)
        
        except Exception as e:
            # Extraction failed, continue without updating
            print(f"  Memory extraction failed: {e}")
            pass
    
    async def invoking(
        self,
        messages: ChatMessage | MutableSequence[ChatMessage],
        **kwargs: Any
    ) -> Context:
        """
        Called BEFORE the agent responds.
        Provide remembered user context to the agent.
        """
        instructions: list[str] = []
        
        # Build context from remembered profile
        if self.profile.name:
            instructions.append(f"The user's name is {self.profile.name}.")
        
        if self.profile.preferred_destinations:
            dests = ", ".join(self.profile.preferred_destinations)
            instructions.append(f"User prefers {dests} destinations.")
        
        if self.profile.budget_level:
            instructions.append(f"User's budget level: {self.profile.budget_level}.")
        
        if self.profile.dietary_restrictions:
            restrictions = ", ".join(self.profile.dietary_restrictions)
            instructions.append(f"User has dietary restrictions: {restrictions}.")
        
        if self.profile.interests:
            interests = ", ".join(self.profile.interests)
            instructions.append(f"User's travel interests: {interests}.")
        
        # Return context with all remembered info
        context_text = " ".join(instructions) if instructions else ""
        return Context(instructions=context_text)
    
    def serialize(self) -> str:
        """Serialize the profile for persistence."""
        return self.profile.model_dump_json()

print(" TravelMemory Context Provider created!")

 TravelMemory Context Provider created!


## Step 5: Agent with Memory - First Conversation

Let's see memory in action!

In [5]:
async def agent_with_memory_first_session():
    """
    First session: Agent learns about the user.
    """
    print("=== Agent With Memory - Learning About User ===")
    
    async with AzureCliCredential() as credential:
        chat_client = AzureAIAgentClient(async_credential=credential)
        
        # Create memory provider
        memory = TravelMemory(chat_client)
        
        # Create agent with memory
        async with ChatAgent(
            chat_client=chat_client,
            name="MemoryEnabledTravelAgent",
            instructions="""
            You are a personalized travel assistant.
            Always use the user's name when you know it.
            Tailor recommendations based on their preferences.
            """,
            context_providers=memory,
        ) as agent:
            thread = agent.get_new_thread()
            
            # Conversation 1: Share name and preferences
            query1 = "Hi! My name is Sarah and I love beach vacations."
            print(f"\nUser: {query1}")
            response1 = await agent.run(query1, thread=thread)
            print(f"Agent: {response1.text}")
            
            # Conversation 2: Share budget
            query2 = "I'm on a moderate budget, nothing too expensive."
            print(f"\nUser: {query2}")
            response2 = await agent.run(query2, thread=thread)
            print(f"Agent: {response2.text}")
            
            # Conversation 3: Share dietary info
            query3 = "Also, I'm vegetarian so food options are important to me."
            print(f"\nUser: {query3}")
            response3 = await agent.run(query3, thread=thread)
            print(f"Agent: {response3.text}")
            
            # Inspect what the memory learned
            print("\n" + "="*60)
            print("MEMORY LEARNED:")
            print("="*60)
            print(f"Name: {memory.profile.name}")
            print(f"Preferred Destinations: {memory.profile.preferred_destinations}")
            print(f"Budget Level: {memory.profile.budget_level}")
            print(f"Dietary Restrictions: {memory.profile.dietary_restrictions}")
            print(f"Interests: {memory.profile.interests}")
            
            # Save memory for next session
            saved_memory = memory.serialize()
            print(f"\n Memory serialized for persistence:")
            print(saved_memory)
            
            return saved_memory

saved_memory = await agent_with_memory_first_session()

=== Agent With Memory - Learning About User ===

User: Hi! My name is Sarah and I love beach vacations.
Agent: Hi Sarah! It’s great to meet you, and beach vacations are always a fantastic choice. To help you find the perfect getaway, could you let me know a little more about your preferences? For example: 

- Are you looking for something relaxing, adventurous, or a mix of both?
- Do you prefer tropical destinations, secluded beaches, or bustling beach towns?
- Any favorite countries or regions?
- Any special interests (snorkeling, surfing, local cuisine, spa treatments, etc.)?

Let me know, and I’ll share personalized beach vacation recommendations just for you!

User: I'm on a moderate budget, nothing too expensive.
Agent: Hi Sarah! It’s great to meet you, and beach vacations are always a fantastic choice. To help you find the perfect getaway, could you let me know a little more about your preferences? For example: 

- Are you looking for something relaxing, adventurous, or a mix of 

## Step 6: Resume with Saved Memory

Now let's create a NEW agent with the saved memory - simulating a new session.

In [6]:
async def agent_with_restored_memory(saved_memory_json: str):
    """
    New session: Agent restores memory and remembers the user!
    """
    print("\n=== Agent With Memory - New Session (Restored Memory) ===")
    print("Simulating a completely new session with saved memory...\n")
    
    async with AzureCliCredential() as credential:
        chat_client = AzureAIAgentClient(async_credential=credential)
        
        # Restore memory from saved state
        profile_dict = json.loads(saved_memory_json)
        restored_profile = TravelProfile.model_validate(profile_dict)
        
        print(" Restored profile:")
        print(f"   Name: {restored_profile.name}")
        print(f"   Preferences: {restored_profile.preferred_destinations}")
        print(f"   Budget: {restored_profile.budget_level}\n")
        
        # Create memory with restored profile
        memory = TravelMemory(chat_client, profile=restored_profile)
        
        # Create NEW agent with restored memory
        async with ChatAgent(
            chat_client=chat_client,
            name="MemoryEnabledTravelAgent_Session2",
            instructions="""
            You are a personalized travel assistant.
            Always use the user's name when you know it.
            Tailor recommendations based on their preferences.
            """,
            context_providers=memory,
        ) as agent:
            # NEW thread (different conversation)
            thread = agent.get_new_thread()
            
            # Ask for recommendation - agent should remember everything!
            query = "Can you recommend a destination for my next trip?"
            print(f"User: {query}")
            response = await agent.run(query, thread=thread)
            print(f"Agent: {response.text}\n")
            
            print("="*60)
            print(" SUCCESS!")
            print("="*60)
            print("Notice how the agent:")
            print("1. Used Sarah's name")
            print("2. Recommended beach destinations")
            print("3. Considered moderate budget")
            print("4. Mentioned vegetarian food options")
            print("\nAll from a DIFFERENT session and thread!")

await agent_with_restored_memory(saved_memory)


=== Agent With Memory - New Session (Restored Memory) ===
Simulating a completely new session with saved memory...

 Restored profile:
   Name: Sarah
   Preferences: ['beach']
   Budget: moderate

User: Can you recommend a destination for my next trip?
Agent: Absolutely, Sarah! Since you love beach destinations, have a moderate budget, and follow a vegetarian diet, here are a few tailored suggestions:

### 1. **Lisbon & Algarve, Portugal**
- **Why?** The southern Algarve region has stunning beaches like Praia da Marinha and Lagos, with charming towns and turquoise waters. Lisbon offers vibrant vegetarian cuisine and easy access to coastal areas.
- **Budget:** Moderate, especially if you book guesthouses or boutique hotels.
- **Vegetarian-friendly:** Portugal has an emerging vegetarian scene—look for “mercearia” (local groceries) and vegetarian restaurants in Lisbon.

### 2. **Nha Trang, Vietnam**
- **Why?** Nha Trang features beautiful sandy beaches, clear waters, and nearby cultural 

## Step 7: Memory Updates Over Time

Memory can evolve as the user shares more information.

In [7]:
async def memory_evolution():
    """
    Demonstrate how memory grows and updates over time.
    """
    print("=== Memory Evolution Over Time ===")
    
    async with AzureCliCredential() as credential:
        chat_client = AzureAIAgentClient(async_credential=credential)
        memory = TravelMemory(chat_client)
        
        async with ChatAgent(
            chat_client=chat_client,
            name="EvolvingMemoryAgent",
            instructions="You are a helpful travel assistant.",
            context_providers=memory,
        ) as agent:
            thread = agent.get_new_thread()
            
            # Week 1: Initial preferences
            print("\n--- Week 1: First Trip ---")
            await agent.run("I'm Tom and I enjoy mountain hiking.", thread=thread)
            print(f"Memory: {memory.profile.model_dump()}\n")
            
            # Week 2: Add more interests
            print("--- Week 2: Discovering More ---")
            await agent.run("I also love history and cultural sites.", thread=thread)
            print(f"Memory: {memory.profile.model_dump()}\n")
            
            # Week 3: Budget information
            print("--- Week 3: Budget Planning ---")
            await agent.run("For this trip, I'm looking at luxury experiences.", thread=thread)
            print(f"Memory: {memory.profile.model_dump()}\n")
            
            # Week 4: New destination type
            print("--- Week 4: Expanding Horizons ---")
            await agent.run("I'm thinking of trying a city break this time.", thread=thread)
            print(f"Memory: {memory.profile.model_dump()}\n")
            
            print(" Memory evolved from basic to rich user profile!")

await memory_evolution()

=== Memory Evolution Over Time ===

--- Week 1: First Trip ---
Memory: {'name': 'Tom', 'preferred_destinations': ['mountain'], 'budget_level': None, 'dietary_restrictions': [], 'interests': ['hiking']}

--- Week 2: Discovering More ---
Memory: {'name': 'Tom', 'preferred_destinations': ['mountain'], 'budget_level': None, 'dietary_restrictions': [], 'interests': ['hiking']}

--- Week 2: Discovering More ---
Memory: {'name': 'Tom', 'preferred_destinations': ['mountain'], 'budget_level': None, 'dietary_restrictions': [], 'interests': ['hiking', 'history', 'cultural sites']}

--- Week 3: Budget Planning ---
Memory: {'name': 'Tom', 'preferred_destinations': ['mountain'], 'budget_level': None, 'dietary_restrictions': [], 'interests': ['hiking', 'history', 'cultural sites']}

--- Week 3: Budget Planning ---
Memory: {'name': 'Tom', 'preferred_destinations': ['mountain'], 'budget_level': 'luxury', 'dietary_restrictions': [], 'interests': ['hiking', 'history', 'cultural sites']}

--- Week 4: Expa

## Step 8: Practical Example - Multi-Session Planning

Real-world scenario: User plans a trip across multiple sessions.

In [8]:
# Add our tools from previous tutorials
def get_weather(
    location: Annotated[str, Field(description="City or country name")],
) -> str:
    """Get current weather for a location."""
    conditions = ["sunny", "partly cloudy", "cloudy", "rainy"]
    temp = randint(15, 32)
    return f"Weather in {location}: {choice(conditions)}, {temp}°C"

def get_attractions(
    location: Annotated[str, Field(description="City or country name")],
) -> str:
    """Get top attractions for a destination."""
    attractions = {
        "Bali": "Beaches, Rice Terraces, Temples, Surfing, Yoga Retreats",
        "Thailand": "Bangkok Temples, Beaches, Markets, Street Food, Islands",
        "Maldives": "Beaches, Diving, Luxury Resorts, Water Sports",
    }
    for city, attrs in attractions.items():
        if city.lower() in location.lower():
            return f"Top attractions in {location}: {attrs}"
    return f"Popular attractions in {location}: Various tourist sites"

print(" Tools defined")

 Tools defined


In [11]:
async def multi_session_planning():
    """
    Realistic multi-session trip planning with persistent memory.
    """
    print("=== Multi-Session Trip Planning ===")
    
    # Session 1: Initial exploration
    print("\n--- SESSION 1: Initial Exploration ---")
    
    async with AzureCliCredential() as credential:
        chat_client = AzureAIAgentClient(async_credential=credential)
        memory = TravelMemory(chat_client)
        
        async with ChatAgent(
            chat_client=chat_client,
            name="TravelPlanner",
            instructions="""
            You are a personalized travel planner.
            Use what you know about the user to make tailored recommendations.
            """,
            tools=[get_weather, get_attractions],
            context_providers=memory,
        ) as agent:
            thread1 = agent.get_new_thread()
            
            query1 = "Hi, I'm Emma. I want to plan a relaxing beach vacation. I'm vegetarian."
            print(f"User: {query1}")
            response = await agent.run(query1, thread=thread1)
            print(f"Agent: {response.text}\n")
    
    # Save memory between sessions
    session1_memory = memory.serialize()
    print(f" Saved memory after session 1")
    print(f"Memory state: {json.loads(session1_memory)}\n")
    
    # Session 2: Research specific destinations (days later)
    print("--- SESSION 2: Researching Destinations (3 days later) ---")
    
    async with AzureCliCredential() as credential:
        chat_client = AzureAIAgentClient(async_credential=credential)
        
        # Restore memory
        profile = TravelProfile.model_validate(json.loads(session1_memory))
        memory = TravelMemory(chat_client, profile=profile)
        print(f" Restored memory: name={profile.name}, preferences={profile.preferred_destinations}")
        
        async with ChatAgent(
            chat_client=chat_client,
            name="TravelPlanner",
            instructions="""
            You are a personalized travel planner.
            Use what you know about the user to make tailored recommendations.
            """,
            tools=[get_weather, get_attractions],
            context_providers=memory,
        ) as agent:
            thread2 = agent.get_new_thread()  # New conversation!
            
            query2 = "I'm thinking about Bali or Maldives. What's the weather like?"
            print(f"\nUser: {query2}")
            response = await agent.run(query2, thread=thread2)
            print(f"Agent: {response.text}\n")
            
            query3 = "I'm on a moderate budget. Which would you recommend?"
            print(f"User: {query3}")
            response = await agent.run(query3, thread=thread2)
            print(f"Agent: {response.text}\n")
    
    # Save updated memory
    session2_memory = memory.serialize()
    print(f" Saved memory after session 2")
    print(f"Memory state: {json.loads(session2_memory)}\n")
    
    # Session 3: Final decision (a week later)
    print("--- SESSION 3: Final Decision (1 week later) ---")
    
    async with AzureCliCredential() as credential:
        chat_client = AzureAIAgentClient(async_credential=credential)
        
        # Restore memory again
        profile = TravelProfile.model_validate(json.loads(session2_memory))
        memory = TravelMemory(chat_client, profile=profile)
        print(f" Restored memory: name={profile.name}, budget={profile.budget_level}")
        
        async with ChatAgent(
            chat_client=chat_client,
            name="TravelPlanner",
            instructions="""
            You are a personalized travel planner.
            Use what you know about the user to make tailored recommendations.
            """,
            tools=[get_weather, get_attractions],
            context_providers=memory,
        ) as agent:
            thread3 = agent.get_new_thread()  # Another new conversation!
            
            query4 = "I've decided on Bali! What are the must-see attractions?"
            print(f"\nUser: {query4}")
            response = await agent.run(query4, thread=thread3)
            print(f"Agent: {response.text}\n")
    
    print("="*60)
    print(" Multi-Session Planning Complete!")
    print("="*60)
    print("Notice:")
    print("- 3 different sessions (days apart)")
    print("- 3 different threads (separate conversations)")
    print("- Agent remembered: Emma's name, beach preference, vegetarian, budget")
    print("- Memory persisted across all sessions!")

await multi_session_planning()

=== Multi-Session Trip Planning ===

--- SESSION 1: Initial Exploration ---
User: Hi, I'm Emma. I want to plan a relaxing beach vacation. I'm vegetarian.
Agent: Hi Emma! That sounds wonderful. To help you plan the perfect relaxing beach vacation with your vegetarian preferences in mind, could you please tell me:

- Do you have any preferred destinations (e.g., Hawaii, Bali, Greece, Caribbean)?
- What is your ideal travel time or month?
- Are you looking for a resort, boutique hotel, or something unique (e.g., eco-lodge)?
- Any specific activities you’d like (spa, yoga, water sports, sightseeing)?
- Will you be traveling solo, with a partner or family?

Let me know your preferences, and I’ll curate recommendations tailored just for you!

Agent: Hi Emma! That sounds wonderful. To help you plan the perfect relaxing beach vacation with your vegetarian preferences in mind, could you please tell me:

- Do you have any preferred destinations (e.g., Hawaii, Bali, Greece, Caribbean)?
- What is 

##  Understanding Context Providers

### Lifecycle of a Context Provider

```python
# 1. User sends message
await agent.run("I love beaches", thread=thread)

# 2. BEFORE AI call: invoking() is called
context = await memory.invoking(messages)
# Returns: Context(instructions="User prefers beaches")

# 3. AI gets enhanced instructions
# Original: "You are a travel assistant"
# Enhanced: "You are a travel assistant. User prefers beaches."

# 4. AI responds with personalized answer

# 5. AFTER AI call: invoked() is called
await memory.invoked(request_messages, response_messages)
# Extracts: beach preference -> stores in memory
```

### Key Methods

| Method | When Called | Purpose |
|--------|-------------|----------|
| `__init__()` | Creation | Initialize memory state |
| `invoking()` | Before AI | Provide context to AI |
| `invoked()` | After AI | Extract & store info |
| `serialize()` | Manual | Save state for persistence |

### Memory vs Thread Storage

**Thread (ChatMessageStore):**
- Stores: Conversation messages
- Lifetime: Per conversation
- Format: List of messages
- Use: Context window for AI

**Memory (ContextProvider):**
- Stores: Extracted facts/preferences
- Lifetime: Across sessions
- Format: Structured data (Pydantic models)
- Use: Personalization & knowledge

##  Key Takeaways

### What We Learned

1. **Threads ≠ Memory**
   - Threads: Conversation history (temporary)
   - Memory: User knowledge (persistent)

2. **Context Providers**
   - Listen to conversations (`invoked()`)
   - Provide context (`invoking()`)
   - Persist state (`serialize()`)

3. **Personalization**
   - Agents remember user across sessions
   - Tailored recommendations based on preferences
   - Evolving understanding over time

4. **Practical Patterns**
   - Save memory to database
   - Restore memory on session start
   - Update memory as user shares more

### Best Practices

1. **Use Pydantic Models** for structured memory
2. **Serialize frequently** to avoid data loss
3. **Validate extracted data** before storing
4. **Combine with threads** for best results
5. **Handle errors gracefully** in extraction

### Production Patterns

```python
# Database integration
class UserMemoryService:
    async def load_memory(self, user_id: str) -> TravelProfile:
        data = await db.get_user_profile(user_id)
        return TravelProfile.model_validate(data)
    
    async def save_memory(self, user_id: str, memory: TravelMemory):
        await db.update_user_profile(user_id, memory.serialize())

# Usage
profile = await memory_service.load_memory(user_id)
memory = TravelMemory(chat_client, profile=profile)

# ... use agent ...

await memory_service.save_memory(user_id, memory)
```

##  Practice Exercises

1. **Extend TravelProfile** - Add fields like preferred_activities, travel_companions, accessibility_needs
2. **Multi-user memory** - Create a memory system that handles multiple users
3. **Memory decay** - Implement time-based memory relevance (older preferences matter less)
4. **Conflict resolution** - Handle contradicting preferences ("I said I hate cold, now I want Alaska?")

In [ ]:
# Exercise: Create an enhanced profile with activity preferences

class EnhancedTravelProfile(BaseModel):
    """Extended travel profile with more details."""
    # Your code here - add more fields!
    pass

# Create a context provider that uses this enhanced profile
# Test it with various user inputs

##  What's Next?

Excellent! You've mastered persistent memory and personalization.

But production agents need more:
-  No content filtering or safety checks
-  No logging or debugging capabilities
-  No request/response transformation

**In Tutorial 05: Middleware and Filters**, you'll learn to:
- Add content filtering for safety
- Implement logging and observability
- Transform requests and responses
- Build production-ready safeguards

---

### Quick Reference

**Create a Context Provider:**
```python
class MyMemory(ContextProvider):
    async def invoked(self, request_messages, response_messages, ...):
        # Extract and store info
        pass
    
    async def invoking(self, messages, ...):
        # Provide context
        return Context(instructions="...")
```

**Use with Agent:**
```python
memory = MyMemory(chat_client)
agent = ChatAgent(..., context_providers=memory)
```

**Persist Memory:**
```python
saved = memory.serialize()
# ... save to database ...
# ... later ...
restored = MyMemory(chat_client, **json.loads(saved))
```